# Airfoil Self-Noise Prediction using PySpark

This project uses the NASA Airfoil Self-Noise dataset to predict the sound pressure level based on several input features using Apache Spark.

In [ ]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [1]:
import warnings
warnings.filterwarnings('ignore')

import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
    .appName("AirfoilNoisePrediction") \
    .getOrCreate()

In [3]:
import os
os.makedirs("/tmp", exist_ok=True)

In [6]:
from pyspark.sql.types import StructType, StructField, DoubleType

# Define schema manually
schema = StructType([
    StructField("Frequency", DoubleType(), True),
    StructField("Angle_of_attack", DoubleType(), True),
    StructField("Chord_length", DoubleType(), True),
    StructField("Free_stream_velocity", DoubleType(), True),
    StructField("Suction_thickness", DoubleType(), True),
    StructField("Sound_pressure_level", DoubleType(), True)
])

# Load the data
file_path = r"C:\Users\pc\Desktop\90DayML\ML_With_Spark\NASA_airfoil_noise_raw.csv"
df = spark.read.csv(file_path, schema=schema, header=False)
df.show(5)

+---------+---------------+------------+--------------------+-----------------+--------------------+
|Frequency|Angle_of_attack|Chord_length|Free_stream_velocity|Suction_thickness|Sound_pressure_level|
+---------+---------------+------------+--------------------+-----------------+--------------------+
|     NULL|           NULL|        NULL|                NULL|             NULL|                NULL|
|    800.0|            0.0|      0.3048|                71.3|       0.00266337|             126.201|
|   1000.0|            0.0|      0.3048|                71.3|       0.00266337|             125.201|
|   1250.0|            0.0|      0.3048|                71.3|       0.00266337|             125.951|
|   1600.0|            0.0|      0.3048|                71.3|       0.00266337|             127.591|
+---------+---------------+------------+--------------------+-----------------+--------------------+
only showing top 5 rows


In [7]:
df.printSchema()
df.describe().show()

root
 |-- Frequency: double (nullable = true)
 |-- Angle_of_attack: double (nullable = true)
 |-- Chord_length: double (nullable = true)
 |-- Free_stream_velocity: double (nullable = true)
 |-- Suction_thickness: double (nullable = true)
 |-- Sound_pressure_level: double (nullable = true)

+-------+------------------+-----------------+-------------------+--------------------+--------------------+--------------------+
|summary|         Frequency|  Angle_of_attack|       Chord_length|Free_stream_velocity|   Suction_thickness|Sound_pressure_level|
+-------+------------------+-----------------+-------------------+--------------------+--------------------+--------------------+
|  count|              1521|             1522|               1521|                1520|                1522|                1522|
|   mean|2876.1505588428668|6.892378449408612|0.13600105193951154|   50.79736842105282| 0.01164257367148486|  124.71772010512468|
| stddev| 3141.146684115996|5.962034806468277|0.09296555558

## Feature Engineering and Vector Assembling

In [8]:
from pyspark.ml.feature import VectorAssembler

# Select features and label
feature_cols = [
    "Frequency", "Angle_of_attack", "Chord_length",
    "Free_stream_velocity", "Suction_thickness"
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
data = assembler.transform(df).select("features", "Sound_pressure_level")
data.show(5)

Py4JJavaError: An error occurred while calling o74.showString.
: org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] User defined function (`VectorAssembler$$Lambda/0x000001e4370ce380`: (struct<Frequency:double,Angle_of_attack:double,Chord_length:double,Free_stream_velocity:double,Suction_thickness:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) failed due to: org.apache.spark.SparkException: Encountered null while assembling a row with handleInvalid = "error". Consider
removing nulls from dataset or using handleInvalid = "keep" or "skip".. SQLSTATE: 39000
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:195)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execu

## Train-Test Split and Model Training

In [9]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol="features", labelCol="Sound_pressure_level")
model = lr.fit(train_data)

Py4JJavaError: An error occurred while calling o82.fit.
: org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] User defined function (`VectorAssembler$$Lambda/0x000001e4370ce380`: (struct<Frequency:double,Angle_of_attack:double,Chord_length:double,Free_stream_velocity:double,Suction_thickness:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) failed due to: org.apache.spark.SparkException: Encountered null while assembling a row with handleInvalid = "error". Consider
removing nulls from dataset or using handleInvalid = "keep" or "skip".. SQLSTATE: 39000
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:195)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.cata

## Model Evaluation

In [10]:
predictions = model.transform(test_data)

from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="Sound_pressure_level",
    predictionCol="prediction",
    metricName="rmse"
)

rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

NameError: name 'model' is not defined

In [ ]:
print("Model Coefficients:", model.coefficients)
print("Model Intercept:", model.intercept)

## Save Final Predictions

In [ ]:
predictions.select("prediction", "Sound_pressure_level").show(10)